# Лабораторная работа 2. Модели представления текстов

In [130]:
import json, time, os, string, re
import numpy as np
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [131]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\artem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\artem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [91]:
ENCODING_DEFAULT = 'utf-8'
TRAIN_FILE = 'train.json'
TEST_FILE = 'test.json'
PRECISION = 4 # the number of signs after the comma

ID = 'id'
TEXT = 'text'
SENTIMENT = 'sentiment'
SENTIMENTS = ['negative', 'positive', 'neutral']
POSITIVE = 0
NEGATIVE = 1
NEUTRAL = 2

ORIGIN_WORD = 'OriginWord'
NORMAL_FORM = 'NormalForm'
POS = 'POS'
CASE = 'Case'
GENDER = 'Gender'
DICT_EXIST = 'FromDict'
WORDS = "Words"
FREQ = "Freq"
UNIQ = "uniquies"
COUNT = "count"

In [ ]:
gr = {
    "NOUN":"имя существительное",
    "ADJF": "имя прилагательное (полное)",
    "NOUN":"имя существительное",
    "ADJF":"имя прилагательное (полное)",
    "ADJS":"имя прилагательное (краткое)",
    "COMP":"компаратив",
    "VERB":"глагол (личная форма)",
    "INFN":"глагол (инфинитив)",
    "PRTF":"причастие (полное)",
    "PRTS":"причастие (краткое)",
    "GRND":"деепричастие",
    "NUMR":"числительное",
    "ADVB":"наречие",
    "NPRO":"местоимение-существительное",
    "PRED":"предикатив",
    "PREP":"предлог",
    "CONJ":"союз",
    "PRCL":"частица",
    "INTJ":"междометие",
    "nomn":"именительный",
    "gent":"родительный",
    "datv":"дательный",
    "accs":"винительный",
    "ablt":"творительный",
    "loct":"предложный",
    "voct":"звательный",
    "gen2":"второй родительный (частичный)",
    "acc2":"второй винительный",
    "loc2":"второй предложный (местный)",
    "masc":"мужской род",
    "femn":"женский род",
    "neut":"средний род",
    "LATN":"Токен состоит из латинских букв (например, “foo-bar” или “Maßstab”)",
    "PNCT":"Пунктуация (например, , или !? или …)",
    "NUMB":"Число (например, “204” или “3.14”)",
    "intg":"целое число (например, “204”)",
    "real":"вещественное число (например, “3.14”)",
    "ROMN":"Римское число (например, XI)",
    "UNKN":"Токен не удалось разобрать",
}

### Задание 1. Подготовка текстового корпуса
- Прочитайте описание задачи Sentiment analysis: https://www.kaggle.com/c/sentiment-analysis-in-russian.
- Скачайте файлы `train.json` и `test.json`: https://www.kaggle.com/c/sentiment-analysis-in-russian/data.
- Загрузите в ноутбук скачанные файлы, выведите информацию по ним.

In [73]:
def read_json(filename):
    start = time.perf_counter()
    with open(filename, 'r', encoding=ENCODING_DEFAULT) as file:
        data = json.load(file)
    print("[INFO]: read time: %0.4f sec" % (time.perf_counter() - start))
    return data

In [86]:
def print_file_description(data, train=False):
    '''размер файла, количество текстов, средняя/макс./мин. длина текста, распределение тональности для обучающей выборки'''
    text_lens = []
    if train:
        sentiments = dict.fromkeys(SENTIMENTS, 0)
    for news in data:
        if train:
            sentiments[news[SENTIMENT]] += 1
        text_lens.append(len(news[TEXT]))
    print("Размер(байты):", os.path.getsize(TRAIN_FILE if train else TEST_FILE))
    print("Количество примеров:", len(data))
    print("Средняя длина текста:", np.mean(text_lens))
    print("Максимальная длина текста:", np.max(text_lens))
    print("Минимальная длина текста:", np.min(text_lens))
    if train:
        print("Распределение тональностей:")
        print("\tПозитивных:", sentiments[SENTIMENTS[POSITIVE]])
        print("\tНегативных:", sentiments[SENTIMENTS[NEGATIVE]])
        print("\tНейтральных:", sentiments[SENTIMENTS[NEUTRAL]])

In [87]:
train_json = read_json(TRAIN_FILE)
print_file_description(train_json, train=True)

[INFO]: read time: 0.4423 sec
Размер(байты): 59298269
Количество примеров: 8263
Средняя длина текста: 3911.85017548106
Максимальная длина текста: 381498
Минимальная длина текста: 28
Распределение тональностей:
	Позитивных: 1434
	Негативных: 2795
	Нейтральных: 4034


In [88]:
test_json = read_json(TEST_FILE)
print_file_description(test_json)

[INFO]: read time: 0.1541 sec
Размер(байты): 15417058
Количество примеров: 2056
Средняя длина текста: 4102.4143968871595
Максимальная длина текста: 320754
Минимальная длина текста: 35


### Задание 2. Булевская модель
Напишите функцию `get_bool_model()` на входе которой текстовый корпус, на выходе – булевская матрица термин-документ.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму;
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса.

Нельзя использовать библиотечные функции `scikit-learn`.  
Можно использовать функции, разработанные в этой и предыдущей лабораторных работах.

In [132]:
morph = pymorphy2.MorphAnalyzer()
nlp = word_tokenize

In [133]:
# tools

def get_data(t):
    return 'None' if t is None else t

def parse_text(text):
    ret = []
    tokens = nlp(text)
    for word in tokens:
        if not word in string.punctuation or len(re.findall(r'^\d+$', word)) == 0:
            parsed = morph.parse(word)[0]
            data = {
                ORIGIN_WORD : word,
                NORMAL_FORM : parsed.normal_form,
                POS         : get_data(parsed.tag.POS),
                CASE        : get_data(parsed.tag.case) ,
                GENDER      : get_data(parsed.tag.gender),
                DICT_EXIST  : morph.word_is_known(word),
            }
            ret.append(data)              
    return ret

def get_dictionary(parsed):
    tmp = {}
    for value in parsed:
        if tmp.get(value[NORMAL_FORM]) is None:
            tmp.update({
            value[NORMAL_FORM] : { 
                POS   : value[POS], 
                WORDS : set(([value[ORIGIN_WORD]])),
                FREQ : 1,
            }})
        else:
            tmp[value[NORMAL_FORM]][WORDS].add(value[ORIGIN_WORD])
            tmp[value[NORMAL_FORM]][FREQ] += 1  
    return tmp

In [116]:
def get_bool_model(data, freq_remove=0, pos_remove=[]):
    '''description'''
    
    return

In [117]:
bool_model = get_bool_model(train_json, freq_remove=2, pos_remove=[])
bool_model

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [ ]:
# Ваш код здесь

### Задание 3. Модель TF-IDF
Напишите функцию `get_tfidf_model()` на входе которой текстовый корпус, на выходе – матрица термин-документ c TF-IDF-весами.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму;
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса;
- вычисляются глобальные IDF-веса и сохраняются в словарь;
- слова для документов взвешиваются в соответствии со схемой TF-IDF.

Нельзя использовать библиотечные функции `scikit-learn`.  
Можно использовать функции, разработанные в предыдущей лабораторной работе.

In [ ]:
def get_tfidf_model():
    # Ваш код здесь
    return

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [ ]:
# Ваш код здесь

Сравните результаты работы вашей функции (полученные веса) с результатами работы класса [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из `scikit-learn`.

In [ ]:
# Ваш код здесь

### Задание 4. Модель word2vec
Напишите функцию `get_word2vec_model()` на входе которой текстовый корпус и модель `word2vec`, на выходе – матрица термин-документ c word2vec-весами.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму (при необходимости – в зависимости от используемой модели);
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса;
- вычисляются веса word2vec для заданной модели.


Можно использовать библиотечные функции и функции, разработанные в лабораторных работах.

In [ ]:
def get_word2vec_model():
    # Ваш код здесь
    return

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [ ]:
# Ваш код здесь